In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import sys

In [ ]:
sys.path.append('/home/gferro/qlm_notebooks/notebooks_1.5.0/Gonzalo/ProgramasDefinitivos/')

In [ ]:
from expectation_module import load_probabilities, load_function, load_U0, load_U1, load_Q
from qat.lang.AQASM import Program, RY, CNOT, H, AbstractGate, QRoutine, SWAP, PH, I
from qat.lang.AQASM.qftarith import QFT

In [ ]:
a = 0.
b = 1.
nqbits = 2
n_aux  = 3 


In [ ]:
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
centers, probs, P_gate = load_probabilities(nqbits, p, a, b)

In [ ]:
R_gate, y = load_function(centers, f, nqbits)

In [ ]:
plt.plot(centers, probs)
plt.plot(centers, y)

In [ ]:
qprog = Program()
qbits = qprog.qalloc(nqbits+1)
qprog.apply(P_gate, qbits[:-1])
qprog.apply(R_gate, qbits)

In [ ]:
Q_gate, U0_gate, U1_gate  = load_Q(nqbits, P_gate, R_gate)

In [ ]:
q_aux = qprog.qalloc(n_aux)

In [ ]:
for i, aux in enumerate(q_aux):
    qprog.apply(H, aux)
    # Apply Q**(2**i) controlled by the auxiliary register
    for _ in range(2**(i)):
        qprog.apply(Q_gate.ctrl(), aux, qbits)


In [ ]:
qprog.apply(QFT(n_aux).dag(), q_aux)

In [ ]:
circuit = qprog.to_circ()

In [ ]:
from qat.qpus import LinAlg

In [ ]:
from qat.qpus import PyLinalg
result = PyLinalg().submit(circuit.to_job(qubits = [i+nqbits+1 for i in range(n_aux)])) 

In [ ]:
from qat.qpus import get_default_qpu

In [ ]:
Def=get_default_qpu()

In [ ]:
result = Def.submit(circuit.to_job(qubits = [i+nqbits+1 for i in range(n_aux)])) 

In [ ]:
states, probabilities = [], []
for res in result:
    states.append(res.state.int) # Store the decimal number that represents each state
    probabilities.append(res.probability) # Store its probability
    print("Aux qbits have value %s (the probability of getting this result is %s)"%(res.state, res.probabil  ity))
